In [1]:
pwd

'/mnt/md0/isshamie/Projects/Mito_Trace/workflow/notebooks/clone_af_dendrograms'

In [2]:
INDIR = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/concat/"
COUNT_INDIR = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_init/donor0/"
OUTDIR = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/concat/sc_af/donor0/"
DONOR = 0
var_type = "init"

In [3]:
from os.path import join, exists, dirname
from glob import glob
import pickle
import mplh.cluster_help as ch
import mplh.fig_utils as fu

import os
import vireoSNP
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
print(vireoSNP.__version__)

import pandas as pd
import seaborn as sns
from vireoSNP import Vireo
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

import seaborn as sns
from mplh import cluster_help as ch

0.5.6


In [4]:
if not exists(OUTDIR):
    os.mkdir(OUTDIR)

## Load cluster results and convert to AF

In [5]:
out_f = join(OUTDIR, f"donor{DONOR}")

if var_type == "init" or var_type == "simpleUnion" or var_type == "prefilterMerge_impute":
    curr_af_f = join(COUNT_INDIR, "af.tsv")
    curr_dp_f = join(COUNT_INDIR, "dp.tsv")
    AF_df = pd.read_csv(curr_af_f, sep="\t", index_col=0).transpose()
    DP_df = pd.read_csv(curr_dp_f, sep="\t", index_col=0).transpose()
elif var_type=="simple":
    curr_ad_f = join(COUNT_INDIR, f"donor{DONOR}.AD.mtx")
    curr_dp_f = join(COUNT_INDIR, f"donor{DONOR}.DP.mtx")
    from src.utils.data_io import wrap_load_mtx_df
    AF_df, DP_df = wrap_load_mtx_df(COUNT_INDIR, prefix=f"donor{DONOR}", 
                                    vcf_prefix=f"donor{DONOR}", inc_af=True, cell_names=True, 
                                    cells_meta= f"cell_labels.donor{DONOR}.txt", cells_meta_sep=',')
elif var_type=="mgatkdonor":
    curr_af_f = join(COUNT_INDIR, f"d{DONOR}.af.tsv")
    curr_dp_f = join(COUNT_INDIR, f"d{DONOR}.coverage.tsv")
    AF_df = pd.read_csv(curr_af_f, sep="\t")
    DP_df = pd.read_csv(curr_dp_f, sep="\t")
    

DP_df.index.name = "Variant"
DP_df.columns.name = "Cell"
AF_df.index.name = "Variant"
AF_df.columns.name = "Cell"


In [6]:
AF_df.to_csv(join(OUTDIR, "af.tsv"), sep="\t")
DP_df.to_csv(join(OUTDIR, "dp.tsv"), sep="\t")

In [7]:
print(AF_df.shape)
print(DP_df.shape)
cells_meta = pd.read_csv(join(INDIR, "cells_meta.tsv"), sep='\t')#.sort_values(["donor", "lineage"])
curr_labels = cells_meta[cells_meta["donor"]==DONOR]
print(f"Number of cells in donor {curr_labels.shape[0]}")

(68, 9023)
(68, 9023)
Number of cells in donor 9023
